In [ ]:
import os
import csv
import sys
import random
import scipy
import numpy as np
from sets import Set


workspace_path = '/home/irashadow/python_workspace/Kaggle_Comp/Santander/'

In [10]:
month_dict_inv = dict()
month_dict_inv[0] = '2015-01'
month_dict_inv[1] = '2015-02'
month_dict_inv[2] = '2015-03'
month_dict_inv[3] = '2015-04'
month_dict_inv[4] = '2015-05'
month_dict_inv[5] = '2015-06'
month_dict_inv[6] = '2015-07'
month_dict_inv[7] = '2015-08'
month_dict_inv[8] = '2015-09'
month_dict_inv[9] = '2015-10'
month_dict_inv[10] = '2015-11'
month_dict_inv[11] = '2015-12'
month_dict_inv[12] = '2016-01'
month_dict_inv[13] = '2016-02'
month_dict_inv[14] = '2016-03'
month_dict_inv[15] = '2016-04'
month_dict_inv[16] = '2016-05'
month_dict_inv[17] = '2016-06'

In [2]:
fitems2index_read = open(workspace_path + '/dict/fitems2index.txt', 'r')


fitems_to_index_dict = dict()


for line in fitems2index_read:
    
    line = line.strip()
    
    features_index = int(line.split('|')[0])
    feature_id = line.split('|')[1]
    feature_item = line.split('|')[2]
    
    if feature_id not in fitems_to_index_dict:
        
       fitems_to_index_dict[feature_id] = dict()
    
       
    fitems_to_index_dict[feature_id][feature_item] = features_index
       
 

fitems2index_read.close()


print fitems_to_index_dict.keys()

['f2', 'f3', 'f4', 'f20', 'f7', 'f9', 'f21', 'f23', 'f18', 'f19', 'f12', 'f13', 'f11', 'f16', 'f17', 'f14', 'f15']


In [6]:
data_train = open(workspace_path + 'input/train_ver2.csv')
test_train = open(workspace_path + 'input/test_ver2.csv')

input_data = [data_train]
input_data.append(test_train)


feat_write = open(workspace_path + '/feat/3_1_cust_nominal.feat', 'w')

line_count = 0
line_skip = 0

cust_feat_dict_tr = dict()

#holdon for featurs:f5, f8, f22(f23 in feat_inx_2)
feat_inx_1 = [2,3,4,7,9,11,12,13,14,15,16,17,18,19,20,21,23]

feat_inx_2 = [2,3,4,7,9,11,12,13,14,15,16,17,18,19,20,22,24]

#for i in range(0,100000):
    #line = data_train.readline().strip().split(',')
for data_file in input_data:
    
    headline = data_file.readline().strip()
    
    for line in data_file:
        line = line.strip().split(',')

        line_count += 1    

        prt_line = ''

        if len(line) == 48 or len(line) == 24:

            cust_id = int(line[1])

            fecha_dato = line[0][:-3]

            prt_line = str(cust_id) + '|' + fecha_dato + '|'

            for feat_inx in feat_inx_1:

                line_item = line[feat_inx].replace(' ','')

                if feat_inx == 10:
                   line_item = line[feat_inx][:-3] 

                if line_item in fitems_to_index_dict['f'+str(feat_inx)]:
                   feat_inx_get = fitems_to_index_dict['f'+str(feat_inx)][line_item]
                   prt_line += str(feat_inx_get) + ':1 '


        elif len(line) == 49 or len(line) == 25:

            cust_id = int(line[1])

            fecha_dato = line[0][:-3]

            prt_line = str(cust_id) + '|' + fecha_dato + '|'

            for feat_inx in feat_inx_2:

                line_item = line[feat_inx].replace(' ','')

                if feat_inx == 10:
                   line_item = line[feat_inx][:-3] 

                if feat_inx == 20:

                    line_item = line[20] + '_' + line[21]

                if feat_inx < 21: 
                      if line_item in fitems_to_index_dict['f'+str(feat_inx)]:
                         feat_inx_get = fitems_to_index_dict['f'+str(feat_inx)][line_item]
                         prt_line += str(feat_inx_get) + ':1 '  

                else:     
                      if line_item in fitems_to_index_dict['f'+str(feat_inx-1)]:
                         feat_inx_get = fitems_to_index_dict['f'+str(feat_inx-1)][line_item]
                         prt_line += str(feat_inx_get) + ':1 '

        feat_write.write(prt_line[:-1] +'\n')           

feat_write.close()



print(line_count)  
print(line_skip)  


14576924
0


In [7]:
#print cust_date_products_dict
feat_read = open(workspace_path + '/feat/3_1_cust_nominal.feat', 'r')

cust_date_nominal_dict = dict()

line_count = 0
#for i in range(0,1000):
    #line = feat_read.readline().strip()
for line in feat_read:
    line = line.strip()
    
    line_count += 1


    cust_id = int(line.split('|')[0])
    item_date = line.split('|')[1]
    features = line.split('|')[2]

    if cust_id not in cust_date_nominal_dict:
       
       cust_date_nominal_dict[cust_id] = dict() 
    
    cust_date_nominal_dict[cust_id][item_date] = np.zeros(500)
   
    for feat_item in features.split(' '):
            
        feat_inx_temp = int(feat_item.split(':')[0])
        cust_date_nominal_dict[cust_id][item_date][feat_inx_temp] = 1
            
print line_count           
feat_read.close()    

14576924


In [17]:
cust_2_labels_dict = dict()

check_line_count = 0


feat_data = []
for date_index_major in range(5,17):
    feat_data_read = open(workspace_path + '/L4_feat_standard/4_7_cust_date_nominal_prod_L4_'+str(month_dict_inv[date_index_major])+'_Model.feat', 'r')
    feat_data_write = open(workspace_path + '/L4_feat_max/4_7_cust_date_nominal_prod_L4_'+str(month_dict_inv[date_index_major])+'_Model.feat', 'w')
      

    for line in feat_data_read:

        line = line.strip()
        check_line_count += 1

        if len(line.split('|')) < 10:

           print line 

        cust_id = int(line.split('|')[0])
        date_note = line.split('|')[1]
        label_tokens = line.split('|')[2]
        added_lastmonth_product_features = line.split('|')[3]
        added_lastexist_product_features = line.split('|')[4]
        product_features = line.split('|')[5]
        nominal_features_pres = line.split('|')[6]
        real_features_pres = line.split('|')[7]
        realvalue_features_pres = line.split('|')[8]
        
        nominal_feature_items = ''
        if cust_id in cust_date_nominal_dict:
           if date_note in cust_date_nominal_dict[cust_id]:   
              for inx_item in list(np.where(cust_date_nominal_dict[cust_id][date_note] == 1))[0]:
                  nominal_feature_items += str(5000+inx_item) + ':1 '
            
        prt_line = ''    
        for i in range(0,6):    
            prt_line += line.split('|')[i] + '|'
        prt_line += nominal_feature_items + '|'    
        for i in range(7,9):    
            prt_line += line.split('|')[i] + '|' 
        feat_data_write.write(prt_line+'\n')    
            
            
    feat_data_read.close()
    feat_data_write.close()
            
         
    

#for i in range(0,23):
    #feat_data[i].close()    
    
#feat_file.close()   
print check_line_count    
#feat_read.close() 

436493


956645


956645
